# Updating basic communication between lib and Bernie firmware

In [1]:
import serial
import sys
import time

In [2]:
import bernielib as bl

In [3]:
def listSerialPorts():
    """ 
    Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [4]:
listSerialPorts()

['COM3', 'COM4']

# Testing basic initialization

In [5]:
ber = bl.robot(cartesian_port_name='COM3', loadcell_port_name='COM4')

In [6]:
ber._readAll(ber.cartesian_port)

''

In [7]:
ber._readAll(ber.loadcell_port)

"Arnie's mobile gripper tool\r\nRev. 2.0, 3/21/2020\r\nPower servo up: P on\r\nPower servo down: P off\r\nRotate servo to 30 degrees: G0 30\r\nServo will actually operate after P on.\r\nTare all load cells: T\r\nTare left load cell: T L\r\nTare right load cell: T R\r\nGet left load cell reading: RL\r\nGet right load cell reading: RR\r\n"

In [8]:
ber._writeAndWait(ber.cartesian_port, 'version', '\r', '')

'Build version: ~11myvar2, Build date: Jul  8 2021 01:28:43, MCU: LPC1769, System Clock: 120MHz\r\n  CNC Build 4 axis\n'

# Testing load cells communications

In [37]:
ber.writeAndWaitLoadCell('RR')

'1.20\r\n'

In [38]:
ber.writeLoadCell('T')

In [39]:
ber.writeAndWaitLoadCell('RR')

'-0.10\r\n'

In [40]:
ber.writeAndWaitLoadCell('RL')

'-0.12\r\n'

# Cartesian movements

In [49]:
ber.writeAndWaitCartesian('G0 Z10 F10000')
ber.writeAndWaitCartesian('M400')

'ok\n'

In [50]:
ber.writeAndWaitCartesian('$H')

'ok\n'

# Homing

In [9]:
ber.robotHome()

In [10]:
ber.robotHome(axis='X')

In [15]:
ber.writeAndWaitCartesian('G28.2 A')

'ok\n'